In [57]:
import pandas as pd
import numpy as np


In [58]:
df = pd.read_csv("agri_energy_merged.csv")
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,state,district,year,month,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
0,0,0,0,assam,baksa,2015,1,NaN,NaN,0.0,0.0,NaN
1,1,1,1,assam,baksa,2015,2,NaN,NaN,0.0,0.0,NaN
2,2,2,2,assam,baksa,2015,3,NaN,NaN,0.0,0.0,NaN
3,3,3,3,assam,baksa,2015,4,NaN,NaN,0.0,0.0,NaN
4,4,4,4,assam,baksa,2015,5,NaN,NaN,0.0,0.0,NaN


In [59]:
df.shape

(84612, 12)

In [61]:
df.isna().sum()

Unnamed: 0.2                          0
Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

In [62]:
df['year'].unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
      dtype=int64)

In [63]:
df['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

# Count of null vlalues for wind energy production
here we have filtered data set for wind energy production. state District and month wise

In [64]:
null_wind_mw_data = df[df['wind_mw'].isnull()]
null_wind_mw_count_by_location_month = null_wind_mw_data.groupby(['state', 'district', 'month']).size().reset_index(name='null_wind_mw_count')
display(null_wind_mw_count_by_location_month.head(12))

,state,district,month,null_wind_mw_count
0,assam,baksa,1,11
1,assam,baksa,2,11
2,assam,baksa,3,11
3,assam,baksa,4,11
4,assam,baksa,5,11
5,assam,baksa,6,11
6,assam,baksa,7,11
7,assam,baksa,8,11
8,assam,baksa,9,11
9,assam,baksa,10,11


In [65]:
null_wind_mw_data =df[df['solar_mw_district'].isnull()]
null_wind_mw_count_by_location_month = null_wind_mw_data.groupby(['state', 'district', 'month']).size().reset_index(name='null_wind_mw_count')
display(null_wind_mw_count_by_location_month.head(12))

,state,district,month,null_wind_mw_count
0,assam,baksa,1,11
1,assam,baksa,2,11
2,assam,baksa,3,11
3,assam,baksa,4,11
4,assam,baksa,5,11
5,assam,baksa,6,11
6,assam,baksa,7,11
7,assam,baksa,8,11
8,assam,baksa,9,11
9,assam,baksa,10,11


In [66]:
df.dtypes

Unnamed: 0.2                        int64
Unnamed: 0.1                        int64
Unnamed: 0                          int64
state                              object
district                           object
year                                int64
month                               int64
solar_mw_district                 float64
wind_mw                           float64
bioenergy_mw                      float64
solar_pumps_installed             float64
estimated_co2_reduction_tonnes    float64
dtype: object

In [67]:
df[df['district'].str.contains('nicobar', case=False, na=False)].shape


(132, 12)

In [68]:
df.isna().sum()

Unnamed: 0.2                          0
Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

In [69]:
df[df['district'].str.contains('nicobar', case=False, na=False)].isna().sum()


Unnamed: 0.2                      0
Unnamed: 0.1                      0
Unnamed: 0                        0
state                             0
district                          0
year                              0
month                             0
solar_mw_district                 0
wind_mw                           0
bioenergy_mw                      0
solar_pumps_installed             0
estimated_co2_reduction_tonnes    0
dtype: int64

In [70]:
nicobar_mask = df['district'].str.lower().eq('nicobar')

print("Nicobar rows:", nicobar_mask.sum())  # should print 132


Nicobar rows: 132


# wind energy imputation for Nicobar
#### Reality applied

- No commercial wind

- Small pilot after 2021

- Only May–September

In [71]:
def fill_wind(row):
    if row['month'] in [5,6,7,8,9]:          # monsoon
        if row['year'] >= 2024:
            return 0.2                      # upper pilot
        elif row['year'] >= 2021:
            return 0.1                      # early pilot
    return 0.0

idx = nicobar_mask & df['wind_mw'].isna()
df.loc[idx, 'wind_mw'] = df.loc[idx].apply(fill_wind, axis=1)


# solar capacity decentralized and centure Nicobar
> ##### Anual Baseline
| Year      | MW  |
|:-----------|-----|
| 2015–17   | 0.2 |
| 2018–20   | 0.5 |
| 2021–23   | 1.2 |
| 2024–25   | 1.5 |


In [72]:
def solar_base(year):
    if year <= 2017: return 0.2
    if year <= 2020: return 0.5
    if year <= 2023: return 1.2
    return 1.5

solar_factor = {
    2:1.2, 3:1.2, 4:1.2,   # Feb–Apr peak
    5:1.05,
    6:0.65, 7:0.65, 8:0.65, # monsoon dip
    9:0.85
}

def fill_solar(row):
    return round(
        solar_base(row['year']) *
        solar_factor.get(row['month'], 1.0),
        3
    )

idx = nicobar_mask & df['solar_mw_district'].isna()
df.loc[idx, 'solar_mw_district'] = df.loc[idx].apply(fill_solar, axis=1)


## Bioenergy (very limited pilots) Nicobar

In [73]:
idx = nicobar_mask & df['bioenergy_mw'].isna()
df.loc[idx, 'bioenergy_mw'] = 0.02


# Solar Pumps (Agriculture – PM-KUSUM) Nicobar

Driven by PM-KUSUM, uptake after 2021.

In [74]:
pump_map = {
    2021: 5,
    2022: 10,
    2023: 15,
    2024: 20,
    2025: 25
}

def fill_pumps(row):
    if row['year'] < 2021:
        return 0.0
    return pump_map[row['year']] / 12

idx = nicobar_mask & df['solar_pumps_installed'].isna()
df.loc[idx, 'solar_pumps_installed'] = df.loc[idx].apply(fill_pumps, axis=1)


# CO₂ Reduction (Derived, not fabricated) Nicobar

> Assumptions:

- 1 MW ≈ 140 MWh/year

- Solar: 0.82 tCO₂/MWh

- Wind: 0.90 tCO₂/MWh

- Bio: 0.75 tCO₂/MWh

In [75]:
def fill_co2(row):
    solar = row['solar_mw_district'] * 140 / 12 * 0.82
    wind  = row['wind_mw'] * 140 / 12 * 0.90
    bio   = row['bioenergy_mw'] * 140 / 12 * 0.75
    return round(solar + wind + bio, 2)

idx = nicobar_mask & df['estimated_co2_reduction_tonnes'].isna()
df.loc[idx, 'estimated_co2_reduction_tonnes'] = df.loc[idx].apply(fill_co2, axis=1)


# Final cheaak of null for Nicobar

In [76]:
df[df['district'].str.lower().eq('nicobar')].isna().sum()


Unnamed: 0.2                      0
Unnamed: 0.1                      0
Unnamed: 0                        0
state                             0
district                          0
year                              0
month                             0
solar_mw_district                 0
wind_mw                           0
bioenergy_mw                      0
solar_pumps_installed             0
estimated_co2_reduction_tonnes    0
dtype: int64

In [77]:
df.isna().sum()

Unnamed: 0.2                          0
Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

> # finally we have filled the nicobar value using realistic context

### lets go for another state and district before we will find null data for all districts

In [78]:
cols_to_check = [
    'solar_mw_district',
    'wind_mw',
    'bioenergy_mw',
    'solar_pumps_installed',
    'estimated_co2_reduction_tonnes'
]

state_district_any_null = (
    df[df[cols_to_check].isna().any(axis=1)]
    [['state', 'district']]
    .drop_duplicates()
    .sort_values(['state', 'district'])
)

state_district_any_null


,state,district
22968,andhra pradesh,adilabad
15708,andhra pradesh,anantapur
792,andhra pradesh,chittoor
2112,andhra pradesh,east godavari
4620,andhra pradesh,guntur
...,...,...
21384,west bengal,pashchim medinipur
42108,west bengal,purba medinipur
42504,west bengal,puruliya
47388,west bengal,south 24 parganas


In [79]:
df.loc[df['wind_mw'].isna(), ['state', 'district']] \
  .drop_duplicates() \
  .sort_values(['state', 'district'])


,state,district
0,assam,baksa
26664,assam,barpeta
53592,assam,bongaigaon
28512,assam,cachar
54912,assam,chirang
...,...,...
21384,west bengal,pashchim medinipur
42108,west bengal,purba medinipur
42504,west bengal,puruliya
47388,west bengal,south 24 parganas


In [80]:
null_map = {}

for col in cols_to_check:
    null_map[col] = (
        df.loc[df[col].isna(), ['state', 'district']]
        .drop_duplicates()
        .sort_values(['state', 'district'])
    )

# Example usage
null_map['solar_mw_district']


,state,district
0,assam,baksa
26664,assam,barpeta
53592,assam,bongaigaon
28512,assam,cachar
54912,assam,chirang
...,...,...
80652,sikkim,west
56232,tripura,dhalai
74580,tripura,north tripura
80256,tripura,south tripura


In [81]:
df[df['solar_mw_district'].isna()] \
  .groupby(['state', 'district']) \
  .size() \
  .reset_index(name='null_count') \
  .sort_values('null_count', ascending=False)


,state,district,null_count
0,assam,baksa,132
114,jammu & kashmir,leh (ladakh),132
116,jammu & kashmir,punch,132
117,jammu & kashmir,rajouri,132
118,jammu & kashmir,ramban,132
...,...,...,...
62,bihar,siwan,132
63,bihar,supaul,132
64,bihar,vaishali,132
65,chandigarh,chandigarh,132


In [82]:
state_null_matrix = (
    df.groupby('state')[cols_to_check]
      .apply(lambda x: x.isna().sum())
      .reset_index()
)

state_null_matrix


,state,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
0,andaman & nicobar island,0,0,0,0,0
1,andhra pradesh,0,0,0,3036,0
2,arunanchal pradesh,0,0,0,0,0
3,assam,3564,3564,0,0,3564
4,bihar,5016,5016,0,0,5016
5,chandigarh,132,132,0,132,132
6,chhattisgarh,2376,2376,2376,2376,2376
7,dadara & nagar havelli,132,132,132,132,132
8,daman & diu,264,264,0,264,264
9,goa,264,264,0,0,264


In [83]:
district_null_matrix = (
    df.groupby(['state', 'district'])[cols_to_check]
      .apply(lambda x: x.isna().sum())
      .reset_index()
)

district_null_matrix


,state,district,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
0,andaman & nicobar island,nicobar,0,0,0,0,0
1,andaman & nicobar island,north & middle andaman,0,0,0,0,0
2,andaman & nicobar island,south andaman,0,0,0,0,0
3,andhra pradesh,adilabad,0,0,0,132,0
4,andhra pradesh,anantapur,0,0,0,132,0
...,...,...,...,...,...,...,...
636,west bengal,pashchim medinipur,0,132,0,0,0
637,west bengal,purba medinipur,0,132,0,0,0
638,west bengal,puruliya,0,132,0,0,0
639,west bengal,south 24 parganas,0,132,0,0,0


In [84]:
fully_missing = (
    df.groupby(['state', 'district'])[cols_to_check]
      .apply(lambda x: x.isna().all())
      .reset_index()
)

fully_missing


,state,district,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
0,andaman & nicobar island,nicobar,False,False,False,False,False
1,andaman & nicobar island,north & middle andaman,False,False,False,False,False
2,andaman & nicobar island,south andaman,False,False,False,False,False
3,andhra pradesh,adilabad,False,False,False,True,False
4,andhra pradesh,anantapur,False,False,False,True,False
...,...,...,...,...,...,...,...
636,west bengal,pashchim medinipur,False,True,False,False,False
637,west bengal,purba medinipur,False,True,False,False,False
638,west bengal,puruliya,False,True,False,False,False
639,west bengal,south 24 parganas,False,True,False,False,False


In [85]:
df[cols_to_check].isna().sum()


solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

In [86]:
# Verify target rows
mask_nma = (
    df['state'].str.lower().str.contains('andaman', na=False) &
    df['district'].str.lower().str.contains('north', na=False)
)

print("North & Middle Andaman rows:", mask_nma.sum())

df.loc[mask_nma].head()


North & Middle Andaman rows: 132


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,state,district,year,month,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
22572,22572,22572,22572,andaman & nicobar island,north & middle andaman,2015,1,0.57,0.0,0.03,0.0,5.72
22573,22573,22573,22573,andaman & nicobar island,north & middle andaman,2015,2,0.69,0.0,0.03,0.0,6.86
22574,22574,22574,22574,andaman & nicobar island,north & middle andaman,2015,3,0.72,0.0,0.03,0.0,7.15
22575,22575,22575,22575,andaman & nicobar island,north & middle andaman,2015,4,0.72,0.0,0.03,0.0,7.15
22576,22576,22576,22576,andaman & nicobar island,north & middle andaman,2015,5,0.60,0.0,0.03,0.0,6.00


In [87]:
# Monthly solar yield factors based on Andaman climate
solar_month_factor = {
    2: 1.15,  # Feb
    3: 1.20,  # Mar
    4: 1.20,  # Apr
    5: 1.00,  # May
    6: 0.65,  # Jun
    7: 0.65,  # Jul
    8: 0.65,  # Aug
    9: 0.70,  # Sep
    10: 0.90, # Oct
    11: 0.95, # Nov
    12: 1.00, # Dec
    1: 0.95   # Jan
}


In [88]:
def solar_base_nm_andaman(year):
    if year <= 2018:
        return 0.6        # minimal infra
    elif year <= 2020:
        return 1.2        # early expansion
    elif year <= 2022:
        return 4.0        # Diglipur floating solar
    elif year <= 2024:
        return 7.0        # distributed + grid
    else:
        return 9.0        # rooftop + PM Surya Ghar


In [89]:
def fill_solar_nm(row):
    base = solar_base_nm_andaman(row['year'])
    factor = solar_month_factor.get(row['month'], 1.0)
    return round(base * factor, 3)


In [90]:
idx = mask_nma & df['solar_mw_district'].isna()
df.loc[idx, 'solar_mw_district'] = df.loc[idx].apply(fill_solar_nm, axis=1)


In [91]:
def fill_wind_nm(row):
    if row['month'] in [6,7,8,9] and row['year'] >= 2022:
        return 0.05      # monitoring / hybrid pilots
    return 0.0


In [92]:
idx = mask_nma & df['wind_mw'].isna()
df.loc[idx, 'wind_mw'] = df.loc[idx].apply(fill_wind_nm, axis=1)


In [93]:
idx = mask_nma & df['bioenergy_mw'].isna()
df.loc[idx, 'bioenergy_mw'] = 0.03


In [94]:
pump_map_nm = {
    2019: 5,
    2020: 10,
    2021: 20,
    2022: 50,
    2023: 90,
    2024: 130,
    2025: 150
}


In [95]:
def fill_pumps_nm(row):
    if row['year'] < 2019:
        return 0.0
    return pump_map_nm[row['year']] / 12


In [96]:
idx = mask_nma & df['solar_pumps_installed'].isna()
df.loc[idx, 'solar_pumps_installed'] = df.loc[idx].apply(fill_pumps_nm, axis=1)


In [97]:
def fill_co2_nm(row):
    solar = row['solar_mw_district'] * 140 / 12 * 0.82
    wind  = row['wind_mw'] * 140 / 12 * 0.90
    bio   = row['bioenergy_mw'] * 140 / 12 * 0.75
    return round(solar + wind + bio, 2)


In [98]:
    idx = mask_nma & df['estimated_co2_reduction_tonnes'].isna()
df.loc[idx, 'estimated_co2_reduction_tonnes'] = df.loc[idx].apply(fill_co2_nm, axis=1)


In [99]:
df.loc[mask_nma].isna().sum()


Unnamed: 0.2                      0
Unnamed: 0.1                      0
Unnamed: 0                        0
state                             0
district                          0
year                              0
month                             0
solar_mw_district                 0
wind_mw                           0
bioenergy_mw                      0
solar_pumps_installed             0
estimated_co2_reduction_tonnes    0
dtype: int64

# Filling data for south andman


In [100]:
south_andaman_mask = (
    (df['state'] == 'andaman & nicobar islands') &
    (df['district'] == 'south Andaman')
)

south_andaman = df.loc[south_andaman_mask]
south_andaman.isna().sum()

Unnamed: 0.2                      0.0
Unnamed: 0.1                      0.0
Unnamed: 0                        0.0
state                             0.0
district                          0.0
year                              0.0
month                             0.0
solar_mw_district                 0.0
wind_mw                           0.0
bioenergy_mw                      0.0
solar_pumps_installed             0.0
estimated_co2_reduction_tonnes    0.0
dtype: float64

In [101]:
def estimate_solar_capacity(year):
    if year <= 2017:
        return 5.1
    elif year <= 2021:
        return 20.0
    else:
        return 28.0

df.loc[south_andaman_mask & df['solar_mw_district'].isna(), 'solar_mw_district'] = (
    df.loc[south_andaman_mask, 'year'].apply(estimate_solar_capacity)
)


In [102]:
df.loc[south_andaman_mask & df['wind_mw'].isna(), 'wind_mw'] = 0.0


In [103]:
def estimate_bioenergy(year):
    if year < 2020:
        return 0.05
    elif year <= 2022:
        return 0.25
    else:
        return 0.6

df.loc[south_andaman_mask & df['bioenergy_mw'].isna(), 'bioenergy_mw'] = (
    df.loc[south_andaman_mask, 'year'].apply(estimate_bioenergy)
)


In [104]:
def estimate_pumps(year):
    if year < 2019:
        return 2
    elif year <= 2022:
        return 10
    else:
        return 25

df.loc[south_andaman_mask & df['solar_pumps_installed'].isna(), 'solar_pumps_installed'] = (
    df.loc[south_andaman_mask, 'year'].apply(estimate_pumps)
)


In [105]:
df.loc[
    south_andaman_mask & df['estimated_co2_reduction_tonnes'].isna(),
    'estimated_co2_reduction_tonnes'
] = (
    df.loc[south_andaman_mask, ['solar_mw_district', 'wind_mw', 'bioenergy_mw']]
      .sum(axis=1) * 1000
)


In [106]:
df.loc[south_andaman_mask].isna().sum()


Unnamed: 0.2                      0.0
Unnamed: 0.1                      0.0
Unnamed: 0                        0.0
state                             0.0
district                          0.0
year                              0.0
month                             0.0
solar_mw_district                 0.0
wind_mw                           0.0
bioenergy_mw                      0.0
solar_pumps_installed             0.0
estimated_co2_reduction_tonnes    0.0
dtype: float64

In [107]:
df.to_csv('agri_energy_merged.csv', index=True)

In [108]:
df.isna().sum()

Unnamed: 0.2                          0
Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

In [109]:
# Group by state & district and count nulls column-wise
null_summary = (
    df
    .groupby(['state', 'district'], as_index=False)
    .agg(lambda x: x.isna().sum())
)

# Keep only rows where at least one null exists
null_summary_filtered = null_summary[
    null_summary.drop(columns=['state', 'district']).sum(axis=1) > 0
]

null_summary_filtered

,state,district,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,year,month,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
3,andhra pradesh,adilabad,0,0,0,0,0,0,0,0,132,0
4,andhra pradesh,anantapur,0,0,0,0,0,0,0,0,132,0
5,andhra pradesh,chittoor,0,0,0,0,0,0,0,0,132,0
6,andhra pradesh,east godavari,0,0,0,0,0,0,0,0,132,0
7,andhra pradesh,guntur,0,0,0,0,0,0,0,0,132,0
...,...,...,...,...,...,...,...,...,...,...,...,...
636,west bengal,pashchim medinipur,0,0,0,0,0,0,132,0,0,0
637,west bengal,purba medinipur,0,0,0,0,0,0,132,0,0,0
638,west bengal,puruliya,0,0,0,0,0,0,132,0,0,0
639,west bengal,south 24 parganas,0,0,0,0,0,0,132,0,0,0


In [110]:
south_andaman_mask = (
    (df['state'] == 'andaman & nicobar island') &
    (df['district'] == 'south andaman')
)

south_andaman = df.loc[south_andaman_mask]

In [111]:
def estimate_solar_capacity(year):
    if year <= 2017:
        return 5.1
    elif year <= 2021:
        return 20.0
    else:
        return 28.0

df.loc[south_andaman_mask & df['solar_mw_district'].isna(), 'solar_mw_district'] = (
    df.loc[south_andaman_mask, 'year'].apply(estimate_solar_capacity)
)


ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
df.loc[south_andaman_mask & df['wind_mw'].isna(), 'wind_mw'] = 0.0


In [55]:
def estimate_bioenergy(year):
    if year < 2020:
        return 0.05
    elif year <= 2022:
        return 0.25
    else:
        return 0.6

df.loc[south_andaman_mask & df['bioenergy_mw'].isna(), 'bioenergy_mw'] = (
    df.loc[south_andaman_mask, 'year'].apply(estimate_bioenergy)
)


ValueError: Must have equal len keys and value when setting with an iterable

In [56]:
def estimate_pumps(year):
    if year < 2019:
        return 2
    elif year <= 2022:
        return 10
    else:
        return 25

df.loc[south_andaman_mask & df['solar_pumps_installed'].isna(), 'solar_pumps_installed'] = (
    df.loc[south_andaman_mask, 'year'].apply(estimate_pumps)
)


ValueError: Must have equal len keys and value when setting with an iterable

In [62]:
df.loc[
    south_andaman_mask & df['estimated_co2_reduction_tonnes'].isna(),
    'estimated_co2_reduction_tonnes'
] = (
    df.loc[south_andaman_mask, ['solar_mw_district', 'wind_mw', 'bioenergy_mw']]
      .sum(axis=1) * 1000
)


In [63]:
df.loc[south_andaman_mask].isna().sum()


Unnamed: 0                        0
state                             0
district                          0
year                              0
month                             0
solar_mw_district                 0
wind_mw                           0
bioenergy_mw                      0
solar_pumps_installed             0
estimated_co2_reduction_tonnes    0
dtype: int64

In [64]:
# Group by state & district and count nulls column-wise
null_summary = (
    df
    .groupby(['state', 'district'], as_index=False)
    .agg(lambda x: x.isna().sum())
)

# Keep only rows where at least one null exists
null_summary_filtered = null_summary[
    null_summary.drop(columns=['state', 'district']).sum(axis=1) > 0
]

null_summary_filtered

,state,district,Unnamed: 0,year,month,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
3,andhra pradesh,adilabad,0,0,0,0,0,0,132,0
4,andhra pradesh,anantapur,0,0,0,0,0,0,132,0
5,andhra pradesh,chittoor,0,0,0,0,0,0,132,0
6,andhra pradesh,east godavari,0,0,0,0,0,0,132,0
7,andhra pradesh,guntur,0,0,0,0,0,0,132,0
...,...,...,...,...,...,...,...,...,...,...
636,west bengal,pashchim medinipur,0,0,0,0,132,0,0,0
637,west bengal,purba medinipur,0,0,0,0,132,0,0,0
638,west bengal,puruliya,0,0,0,0,132,0,0,0
639,west bengal,south 24 parganas,0,0,0,0,132,0,0,0


In [65]:
df.isna().sum()

Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 26004
wind_mw                           52140
bioenergy_mw                       6468
solar_pumps_installed             12936
estimated_co2_reduction_tonnes    26004
dtype: int64

In [ ]:
df.to_csv('agri_